In [1]:
# 1 
# Importing all the required libraries 
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [ ]:
# 20
# Scrape the text and labels from the dataset
# insert script to scrape the text and labels. Ask Praneet to combine both scripts to one. TODO
# Also, write multiple files. Plain Text, Stemmed Text, Labels


In [2]:
# 30
# Number of files without English Text. TODO: Rakesh
# Read the scraped file content into a data frame and then remove the files which have text
# There is no English version of this document available since it was not included in the English Special Edition.

_stem_text_df = pd.read_csv("../data/Stem_Text.csv", encoding="latin-1")
# Remove the files without any Eglish text (feature vector)
_stem_text_df["English"] = _stem_text_df["Text"].str.contains('english version document avail sinc includ english special edit', regex=True)
_stem_text_df['English'].value_counts()
# TRUE files do not have English Text in them


False    19411
True       189
Name: English, dtype: int64

In [ ]:
# 40
# How many files have how many number of labels? From Rabi
def count_number(labels):
    split_labels = str(labels).split(',')
    total_labels = len(split_labels)
    return total_labels


if __name__ == "__main__":
    _label_df = pd.read_csv("../data/Eurovoc_Labels.csv")
    # Count number of labels present in each file
    _label_df = _label_df.groupby(['Filename'])['Eurovoc_Label'].apply(', '.join).reset_index()
    _label_df['count_of_labels'] = _label_df['Eurovoc_Label'].apply(count_number)
    # Checking maximum count of label present
    count_max_label = _label_df['count_of_labels'].max()
    index_max = _label_df['count_of_labels'].idxmax()
    print(count_max_label)
    print(_label_df['Filename'].iloc[index_max])
    # Top 10 files with highest number of labels
    # max_label_files = _label_df.nlargest(10, 'count_of_labels').reset_index()
    # plt.bar(max_label_files['Filename'], max_label_files['count_of_labels'])
    # plt.show()

    # Group by count_of_labels and create a new data-frame
    _group_by_labels_df = _label_df.groupby('count_of_labels')['Filename'].apply(', '.join).reset_index()
    _group_by_labels_df['count_of_Filename'] = _group_by_labels_df['Filename'].apply(count_number)
    _group_by_labels_df = _group_by_labels_df.sort_values('count_of_Filename').reset_index()
    _group_by_labels_df = _group_by_labels_df.drop(columns='Filename')
    #print(_group_by_labels_df)

    # Number of labels vs Number of files
    plt.barh(_group_by_labels_df['count_of_labels'].tail(5), _group_by_labels_df['count_of_Filename'].tail(5))
    plt.xlabel('Number of Files')
    plt.ylabel('Number of Labels')
    plt.show()

    # Number of files vs Number of labels
    plt.bar(_group_by_labels_df['count_of_labels'], _group_by_labels_df['count_of_Filename'])
    plt.xlabel('Number of Labels')
    plt.ylabel('Number of Files')
    plt.show()

In [3]:
# 50
# Reading from labels file and transposing them. One row each for a file. 
# Transpose using pivot table. So, we have as many columns as distinct labels. Value under a label shows whether the document has the label or not.
_labels_df = pd.read_csv("../data/Eurovoc_Labels.csv", encoding="latin-1")
_labels_df['pivotcol'] = 1
_lbl_trnsps_df = pd.pivot_table(_labels_df,values='pivotcol',index=['Filename'],columns=['Eurovoc_Label'],aggfunc=np.sum, fill_value=0)
#_lbl_trnsps_df.head()

In [4]:
# 55
# Reading from labels file and transposing them. One row each for a file. 
# Aggregate labels into aggregated list format for each file
_labels_df = _labels_df.groupby(['Filename'])['Eurovoc_Label'].apply(list).reset_index()
type(_lbl_trnsps_df)
type(_labels_df)

pandas.core.frame.DataFrame

In [5]:
# 60
# merge the features and labels transposed 
df_for_stemming = _stem_text_df[_stem_text_df['English']==False]
_stem_text_df = df_for_stemming.drop(columns='English')
_merged_df = pd.merge(_stem_text_df, _lbl_trnsps_df, on='Filename')
_stem_text_df = df_for_stemming.drop(columns='English')
#_merged_df.head()

In [6]:
# 61
# merge the features and array of labels
_array_df = pd.merge(_stem_text_df, _labels_df, on='Filename')
#_array_df.head()
type(_array_df)

pandas.core.frame.DataFrame

In [41]:
testdf = _array_df.head(100)
testdf.head()
type(testdf)

pandas.core.frame.DataFrame

In [42]:
#testdf.to_csv("Testdf.csv")
test1df = pd.read_csv("Testdf_text_tolist.csv",encoding="latin-1")

In [43]:
type(test1df)

pandas.core.frame.DataFrame

In [47]:
# Testing on a subset of data
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(testdf.Eurovoc_Label)
Y = multilabel_binarizer.transform(testdf.Eurovoc_Label)

count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(testdf.Text)

tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)

In [48]:
type(X_tfidf)

scipy.sparse.csr.csr_matrix

In [53]:
testdf.head()

,Filename,Text,Eurovoc_Label
0,32002D0182_EN_NOT.html,commiss decis 28 februari 2002 approv amend pl...,"[animal plague, swine, region, Austria, wild m..."
1,32005R1522_EN_NOT.html,commiss regul ( EC ) 1522/2005 20 septemb 2005...,"[barley, intervention agency, Hungary, award o..."
2,31962R0049_EN_NOT.html,regul 49 council amend date certain instrument...,"[common agricultural policy, common organisati..."
3,32005D0738_EN_NOT.html,commiss decis 14 septemb 2005 clearanc account...,"[EAGGF Guarantee Section, closing of accounts,..."
4,31998D0273_EN_NOT.html,commiss decis 28 januari 1998 relat proceed ar...,"[concessionnaire, fine, infringement of Commun..."


In [55]:
X_tfidf.toarray().shape

(100, 13523)

In [49]:
# 80. this is in [8]
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=71)

In [50]:
X_tfidf_resampled, Y_tfidf_resampled = ros.fit_sample(X_tfidf, Y)

In [37]:
# 90 this is in [9]
x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf_resampled, Y_tfidf_resampled, test_size=0.2, random_state=71)

In [23]:
# 100 this is in [10]. 
# this may not be needed after all. Just plotting
#fig = plt.figure(figsize=(20,20))
#(ax_test, ax_train) = fig.subplots(ncols=2, nrows=1)
#g1 = sns.barplot(x=Y.sum(axis=0), y=multilabel_binarizer.classes_, ax=ax_test)
#g2 = sns.barplot(x=y_train_tfidf.sum(axis=0), y=multilabel_binarizer.classes_, ax=ax_train)
#g1.set_title("class distribution before resampling")
#g2.set_title("class distribution in training set after resampling")

In [38]:
# 110. in [11]
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import hamming_loss

def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float(len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_test_tfidf)))
    print("Hamming score: {}".format(hamming_score(y_pred, y_test_tfidf)))
    print("---")    

In [39]:
# 120. in [12]
nb_clf = MultinomialNB()
#sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=6, tol=None)
#lr = LogisticRegression()
#mn = MultinomialNB()
clf = OneVsRestClassifier(nb_clf)
clf.fit(x_train_tfidf, y_train_tfidf)
y_pred = clf.predict(x_test_tfidf)
print_score(y_pred, nb_clf)

Clf:  MultinomialNB
Hamming loss: 0.013888888888888888
Hamming score: 0.0
---


In [40]:
#nb_clf = MultinomialNB()
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=6, tol=None)
clf = OneVsRestClassifier(sgd)
clf.fit(x_train_tfidf, y_train_tfidf)
y_pred = clf.predict(x_test_tfidf)
print_score(y_pred, sgd)
# https://xang1234.github.io/multi-label/
# Something to try https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/

C:\Users\Rakesh\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


Clf:  SGDClassifier
Hamming loss: 0.0011574074074074073
Hamming score: 0.9166666666666666
---


In [ ]:
# To get rid of the warning, use something like max_iter=5, tol=-np.infty

In [56]:
x1 = X_tfidf

In [57]:
from sklearn.decomposition import TruncatedSVD

In [59]:
svd = TruncatedSVD(n_components=1000)
x2 = svd.fit_transform(x1)

In [64]:
x2

array([[ 0.298459  ,  0.11659759, -0.20156569, ...,  0.01261915,
        -0.28756359,  0.07311586],
       [ 0.33053532, -0.11709454, -0.19780074, ..., -0.00843373,
         0.00577525, -0.01820458],
       [ 0.35505358, -0.08997341, -0.03499974, ..., -0.00829571,
         0.0073518 , -0.01762961],
       ...,
       [ 0.31851803, -0.07073645, -0.1706718 , ..., -0.00303012,
        -0.00097718, -0.00805816],
       [ 0.33616746, -0.15982073, -0.21002734, ...,  0.00197359,
        -0.02447071, -0.01516797],
       [ 0.2832784 ,  0.22831058,  0.17775419, ..., -0.00635583,
         0.00732604,  0.00113425]])